<a href="https://colab.research.google.com/github/hyj12345/hyj12345/blob/main/BIS638_HW4_Ontology.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#**Homework 4** 

## **1.Modify the retrieval code**

In [5]:
import requests
import pandas as pd
# search for DOID terms that have ICD9CM codes
# api-endpoint 
URL = "https://www.ebi.ac.uk/ols/api/search"
# param values here 
searchterm = 'ICD9CM'
ont = "doid"
rowmax = 500  # limit to 20 terms
fieldlist = 'iri,label,obo_id'

# defining a params dict for the parameters to be sent to the API 
PARAMS = {'q': searchterm, 'ontology': ont, 'rows': rowmax, 'fieldList': fieldlist}

# sending get request and saving the response as response object 
r = requests.get(url=URL, params=PARAMS)

# extracting data in json format 
data = r.json()
print(data)
#ontology_name: (doid)'obo_id': 'DOID:13575'  'label': 'non-renal secondary hyperparathyroidism'}
irilist=[]
lablelist=[]
obidlist=[]
for d in data['response']['docs']:
  irilist.append(d['iri'])
  lablelist.append(d['label'])
  obidlist.append(d['obo_id'].replace('DOID:', ''))

data1 = {'iri': irilist,'DOID':obidlist,'label': lablelist}
df1 = pd.DataFrame(data1)



{'responseHeader': {'status': 0, 'QTime': 32, 'params': {'facet.field': ['ontology_name', 'ontology_prefix', 'type', 'subset', 'is_defining_ontology', 'is_obsolete'], 'hl': 'true', 'fl': 'iri,label,obo_id', 'start': '0', 'fq': ['ontology_name: (doid)', 'is_obsolete:false'], 'rows': '500', 'hl.simple.pre': '<b>', 'bq': 'type:ontology^10.0 is_defining_ontology:true^100 label_s:"icd9cm"^5 synonym_s:"icd9cm"^3 annotations_trimmed:"icd9cm"', 'q': 'ICD9CM', 'defType': 'edismax', 'hl.simple.post': '</b>', 'qf': 'label^5 synonym^3 description short_form^2 obo_id^2 annotations logical_description iri', 'hl.fl': ['label', 'synonym', 'definition'], 'facet': 'true', 'wt': 'json'}}, 'response': {'numFound': 2113, 'start': 0, 'docs': [{'iri': 'http://purl.obolibrary.org/obo/DOID_13575', 'obo_id': 'DOID:13575', 'label': 'non-renal secondary hyperparathyroidism'}, {'iri': 'http://purl.obolibrary.org/obo/DOID_9754', 'obo_id': 'DOID:9754', 'label': 'ganglion or cyst of synovium/tendon/bursa'}, {'iri': '

In [6]:
import pandas as pd
from tqdm import tqdm,trange
# the code below retrieve icd9 code for each iri
icdlist = []
parentlist = []
irilist2 = []
synonymlist=[]
doidlist=[]
for i in trange(len(irilist)):
    iri=irilist[i]
    iri2 = iri.replace(':', '%253A')
    iri2 = iri2.replace('/', '%252F')
    
    URL2 = 'https://www.ebi.ac.uk/ols/api/ontologies/' + ont + '/terms/' + iri2
    # print('URL2', URL2)
    r2 = requests.get(url=URL2)
    data2 = r2.json()
    #print('iri', iri)
    #synonym
    try:
        synonymlist.append(data2['annotation']['has_exact_synonym'])
    except KeyError:
        synonymlist.append('[null]')
    #DOID
    doidlist.append(data2['obo_id'].replace('DOID:', ''))

    #icd9
    for d2 in data2['obo_xref']:
        if d2['database'] == 'ICD9CM':
            icd_code = d2['id'].replace('.', '')
            icdlist.append(icd_code)
            break

data2 = {'iri': irilist,'DOID':doidlist,'ICD9': icdlist,'Synonym':synonymlist}
df2 = pd.DataFrame(data2)

100%|██████████| 500/500 [03:40<00:00,  2.27it/s]

                                             iri  ...                                            Synonym
0      http://purl.obolibrary.org/obo/DOID_13575  ...                                             [null]
1       http://purl.obolibrary.org/obo/DOID_9754  ...                                             [null]
2      http://purl.obolibrary.org/obo/DOID_11283  ...                                             [null]
3      http://purl.obolibrary.org/obo/DOID_11129  ...                                             [null]
4       http://purl.obolibrary.org/obo/DOID_8646  ...                                             [null]
..                                           ...  ...                                                ...
495    http://purl.obolibrary.org/obo/DOID_11547  ...                                [Deposits - cornea]
496    http://purl.obolibrary.org/obo/DOID_11541  ...  [recurrent erosion of cornea, recurrent erosio...
497  http://purl.obolibrary.org/obo/DOID_0080176  ...  

## **2.Generate two csv files**

i) DOID to ICD9 code mappings 

ii) synonyms file containing the DOID column and Synonym column. 

In [33]:
df2_=df2[['iri','ICD9']]
d_icd9=pd.merge(df1,df2_,on='iri').drop(['iri'],axis=1)
d_sy=df2.drop(['ICD9','iri'],axis=1)
#Tabel one DOID_ICD9
d_icd9.to_csv('/content/sample_data/DOID_ICD9.csv', index=False)
print('DOID_ICD9 features are {}'.format(doid_icd9.columns))
d_sy.to_csv('/content/sample_data/DOID_Synonym.csv', index=False)
print('DOID_Synonym features are {}'.format(d_sy.columns))

DOID_ICD9 features are Index(['DOID', 'label', 'ICD9'], dtype='object')
DOID_Synonym features are Index(['DOID', 'Synonym'], dtype='object')


## **3.Add two tables to the mimic database. Load the two csv files into these new tables**

In [2]:
from google.colab import drive
drive.mount('/content/gdrive')

Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).


In [ ]:
CREATE TABLE Patients (
    row_id int  NOT NULL,
    subject_id int  NOT NULL,
    gender varchar(5)  NOT NULL,
    dob datetime  NOT NULL,
    dod datetime  NOT NULL,
    dod_hosp datetime  NULL,
    dod_ssn datetime  NULL,
    expire_flag varchar(5)  NOT NULL,
    PRIMARY KEY (subject_id)
  );

CREATE TABLE Admissions (
    row_id int  NOT NULL,
    subject_id int  NOT NULL,
    hadm_id int  NOT NULL,
    admittime datetime  NOT NULL,
    dischtime datetime  NOT NULL,
    deathtime datetime   NULL,
    admission_type varchar(50)  NOT NULL,
    admission_location varchar(50)  NOT NULL,
    discharge_location varchar(50)  NOT NULL,
    insurance varchar(255)  NOT NULL,
    language varchar(10)   NULL,
    religion varchar(50)   NULL,
    marital_status varchar(50)   NULL,
    ethnicity varchar(200)  NOT NULL,
    edregtime datetime   NULL,
    edouttime datetime   NULL,
    diagnosis varchar(300)  NOT NULL,
    hospital_expire_flag int  NOT NULL,
    has_chartevents_data int  NOT NULL,
    PRIMARY KEY (hadm_id),
    FOREIGN KEY (subject_id) REFERENCES Patients(subject_id)
  );

# insert code for creating D_icd_diagnoses 
CREATE TABLE D_icd_diagnoses (
    row_id int  NOT NULL,
    icd9_code varchar(10)  NOT NULL,
    short_title varchar(50)  NOT NULL,
    long_title varchar(300)  NOT NULL,
    PRIMARY KEY (icd9_code)
  );

# Insert code for creating Diagnoses_icd tables
CREATE TABLE Diagnoses_icd tables (
    row_id int  NOT NULL,
    subject_id int  NOT NULL,
    ham_id int  NOT NULL,
    seq_num int  NOT NULL,
    icd9_code varchar(10)  NOT NULL
    PRIMARY KEY (row_id),
    FOREIGN KEY (icd9_code) REFERENCES D_icd_diagnoses(icd9_code),
    FOREIGN KEY (subject_id) REFERENCES Patients(subject_id)

  );

# Insert code for creating DOID_ICD9 tables 'DOID':obidlist,'label': lablelist
CREATE TABLE DOID_ICD9 tables (
    DOID varchar(50)  NOT NULL,
    label varchar(300)  NOT NULL
    PRIMARY KEY (DOID)
  );
# Insert code for creating Diagnoses_icd tables 'DOID':doidlist,'ICD9': icdlist,'Synonym':synonymlist
CREATE TABLE DOID_Synonym tables (
    DOID varchar(50)  NOT NULL,
    ICD9 varchar(50)  NOT NULL,
    Synonym varchar(300)
    PRIMARY KEY (DOID),
    FOREIGN KEY (DOID) REFERENCES DOID_ICD9(DOID),
    FOREIGN KEY (ICD9) REFERENCES D_icd_diagnoses(icd9_code)

  );

## **4.SQL join query to retrieve patients with a certain diagnosis using its synonyms**

In [30]:
import pandas as pd
import sqlite3
from sqlite3 import Error
from IPython.display import display


""" create a database connection to a database that resides
    in the drive
"""
path='/content/gdrive/My Drive/'
conn = None;
try:
  conn = sqlite3.connect(':memory:')
  #conn = sqlite3.connect(path+'mimic_demo.db')

  cursor = conn.cursor()
  #Doping tables if already exists.
  cursor.execute("DROP TABLE IF EXISTS Patients")
  cursor.execute("DROP TABLE IF EXISTS Admissions")
  cursor.execute("DROP TABLE IF EXISTS Diagnoses_icd")
  cursor.execute("DROP TABLE IF EXISTS D_icd_diagnoses")
  cursor.execute("DROP TABLE IF EXISTS DOID_ICD9")
  cursor.execute("DROP TABLE IF EXISTS DOID_Synonym")
  #Creating tables
  sql ='''CREATE TABLE Patients (
    row_id int  NOT NULL,
    subject_id int  NOT NULL,
    gender varchar(5)  NOT NULL,
    dob datetime  NOT NULL,
    dod datetime  NOT NULL,
    dod_hosp datetime  NULL,
    dod_ssn datetime  NULL,
    expire_flag varchar(5)  NOT NULL,
    PRIMARY KEY (subject_id)
  );'''
  cursor.execute(sql)
  sql ='''CREATE TABLE Admissions (
    row_id int  NOT NULL,
    subject_id int  NOT NULL,
    hadm_id int  NOT NULL,
    admittime datetime  NOT NULL,
    dischtime datetime  NOT NULL,
    deathtime datetime   NULL,
    admission_type varchar(50)  NOT NULL,
    admission_location varchar(50)  NOT NULL,
    discharge_location varchar(50)  NOT NULL,
    insurance varchar(255)  NOT NULL,
    language varchar(10)   NULL,
    religion varchar(50)   NULL,
    marital_status varchar(50)   NULL,
    ethnicity varchar(200)  NOT NULL,
    edregtime datetime   NULL,
    edouttime datetime   NULL,
    diagnosis varchar(300)  NOT NULL,
    hospital_expire_flag int  NOT NULL,
    has_chartevents_data int  NOT NULL,
    PRIMARY KEY (hadm_id),
    FOREIGN KEY (subject_id) REFERENCES Patients(subject_id)
  );'''
  cursor.execute(sql)
  #insert code to create D_icd_diagnoses table
  sql='''CREATE TABLE D_icd_diagnoses (
    row_id int  NOT NULL,
    icd9_code varchar(10)  NOT NULL,
    short_title varchar(50)  NOT NULL,
    long_title varchar(300)  NOT NULL,
    PRIMARY KEY (icd9_code)
  );'''
  cursor.execute(sql)
  #insert code to create Diagnoses_icd table
  sql='''
   CREATE TABLE Diagnoses_icd tables (
     row_id int  NOT NULL,
     subject_id int  NOT NULL,
     ham_id int  NOT NULL,
     seq_num int  NOT NULL,
     icd9_code varchar(10)  NOT NULL
     PRIMARY KEY (row_id),
     FOREIGN KEY (icd9_code) REFERENCES D_icd_diagnoses(subject_id),
     FOREIGN KEY (subject_id) REFERENCES Patients(subject_id)
   );'''

   # Insert code for creating DOID_ICD9 tables 'DOID':obidlist,'label': lablelist
  sql='''
   CREATE TABLE DOID_ICD9 tables (
    DOID varchar(50)  NOT NULL,
    label varchar(300)  NOT NULL
    PRIMARY KEY (DOID)
  );'''
# Insert code for creating Diagnoses_icd tables 'DOID':doidlist,'ICD9': icdlist,'Synonym':synonymlist
  sql='''
   CREATE TABLE DOID_Synonym tables (
    DOID varchar(50)  NOT NULL,
    ICD9 varchar(50)  NOT NULL,
    Synonym varchar(300)
    PRIMARY KEY (DOID),
    FOREIGN KEY (DOID) REFERENCES DOID_ICD9(DOID),
    FOREIGN KEY (ICD9) REFERENCES D_icd_diagnoses(icd9_code)
  );'''



  # cursor.execute(sql)


  print("Table(s) created successfully........")
  #conn.commit()

  #load the data from a csv file located in a directory (use your own directory into a Pandas DataFrame object
  patients = pd.read_csv(path+'PATIENTS.csv')
  admissions = pd.read_csv(path+'ADMISSIONS.csv')
  D_icd_diagnoses = pd.read_csv(path+'D_ICD_DIAGNOSES.csv')
  Diagnoses_icd = pd.read_csv(path+'DIAGNOSES_ICD.csv')
  doid_icd9 = pd.read_csv(path+'DOID_ICD9.csv')
  doid_synonym = pd.read_csv(path+'DOID_Synonym.csv')


  #write the data to a sqlite table
  patients.to_sql('Patients', conn, if_exists='replace', index = False)
  admissions.to_sql('Admissions', conn, if_exists='replace', index = False)
  D_icd_diagnoses.to_sql('D_icd_diagnoses', conn, if_exists='replace', index = False)
  Diagnoses_icd.to_sql('Diagnoses_icd', conn, if_exists='replace', index = False)
  doid_icd9.to_sql('DOID_ICD9', conn, if_exists='replace', index = False)
  doid_synonym.to_sql('DOID_Synonym', conn, if_exists='replace', index = False)

  #Commit your changes in the database
  conn.commit()
 
 
  sql='''SELECT distinct sy.DOID,ICD9,short_title,long_title,Synonym,p.subject_id,gender,admittime,deathtime,
  admission_type,admission_location,discharge_location,insurance,language,religion,marital_status,ethnicity
  FROM doid_synonym sy inner join doid_icd9 dicd9 on sy.DOID=dicd9.DOID
  inner join D_icd_diagnoses D_icd on dicd9.ICD9=D_icd.icd9_code
  inner join Diagnoses_icd D_i on D_icd.icd9_code=D_i.icd9_code
  inner join Patients p on D_i.subject_id=p.subject_id 
  inner join Admissions a on a.subject_id=p.subject_id
  where  Synonym LIKE '%rectum%'
  '''
 
  patients = pd.read_sql(sql, conn)
  display (patients)
  

except Error as e:
    print(e)
finally:
    if conn:
        conn.close()


Table(s) created successfully........


,DOID,ICD9,short_title,long_title,Synonym,subject_id,gender,admittime,deathtime,admission_type,admission_location,discharge_location,insurance,language,religion,marital_status,ethnicity
0,9307,5691,Rectal prolapse,Rectal prolapse,"['Procidentia, rectum']",10027,F,2190-07-13 07:15:00,None,ELECTIVE,PHYS REFERRAL/NORMAL DELI,SNF,Medicare,None,CATHOLIC,MARRIED,WHITE
